In [1]:
import pandas as pd

# 1. Data Reading

In [2]:
data = pd.read_csv("data/Churn_Modelling.csv")

In [3]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
data.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

# 2. Reformat data

In [5]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
data = data.drop(['CustomerId','RowNumber','Surname'],axis=1)
data["Geography"] = lbl.fit_transform(data['Geography'])
data["Gender"] = lbl.fit_transform(data['Gender'])

In [6]:
data.isna().any()

CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
Exited             False
dtype: bool

In [7]:
data = data.iloc[:,3:]

In [8]:
data

,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,42,2,0.00,1,1,1,101348.88,1
1,41,1,83807.86,1,0,1,112542.58,0
2,42,8,159660.80,3,1,0,113931.57,1
3,39,1,0.00,2,0,0,93826.63,0
4,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...
9995,39,5,0.00,2,1,0,96270.64,0
9996,35,10,57369.61,1,1,1,101699.77,0
9997,36,7,0.00,1,0,1,42085.58,1
9998,42,3,75075.31,2,1,0,92888.52,1


# 3. Train test split

In [9]:
from sklearn.model_selection import train_test_split
input_data = data.iloc[:,:-1]
output_data = data.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(input_data,output_data,test_size=0.3,random_state=220)

# 4. Modeling

In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0,class_weight="balanced")

In [11]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=0)

In [12]:
clf.score(X_train,y_train)

1.0

In [13]:
clf.score(X_test,y_test)

0.79

In [14]:
from sklearn.metrics import classification_report as report
preds = clf.predict(X_test)
print(report(y_test,preds,digits=4))

              precision    recall  f1-score   support

           0     0.8648    0.8713    0.8680      2378
           1     0.4934    0.4791    0.4861       622

    accuracy                         0.7900      3000
   macro avg     0.6791    0.6752    0.6771      3000
weighted avg     0.7878    0.7900    0.7889      3000



In [15]:
from sklearn.metrics import classification_report as report
preds = clf.predict(X_test)
print(report(y_test,preds,digits=4))

              precision    recall  f1-score   support

           0     0.8648    0.8713    0.8680      2378
           1     0.4934    0.4791    0.4861       622

    accuracy                         0.7900      3000
   macro avg     0.6791    0.6752    0.6771      3000
weighted avg     0.7878    0.7900    0.7889      3000



In [16]:
from sklearn.tree import export_graphviz
dot_data = export_graphviz(clf, 
                   feature_names=X_train.columns,  
                   class_names=["not_churn","churn"],
                   filled=True)

In [21]:
import os
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'

In [18]:
!pip install graphviz

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import graphviz
graph = graphviz.Source(dot_data, format="png")

graph

In [23]:
fie_ext = 'png'
temp_img = 'temp_file'
graph.render(temp_img,format=fie_ext, view=False)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.372132 to fit


'temp_file.png'

In [ ]:
### It is too much overfitted, what should we do?

In [ ]:
from sklearn.metrics import classification_report as report
preds_ = rdf.predict(X_test)
print(report(y_test,preds_,digits=4))